# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Assessment of Results: Does race influence a job candidate's chances of receiving a call back

Race and callback status are categorical variables. Proving a relationship requires a different technique that using common correlation coefficients. The standard technique for proving a relationship is to use a contingency table, often refered to as a crosstabulation or crosstab, and calculate a Chi-square ($X$^2) statistic for use in testing for statistical significance. If the statistic is statistically significant, then you can conclude that the values of the two variables in the table are dependent upon each other. In this context, a statistically significant result would indicate that receiving a call back from a potentially employer is dependent on have a white-sounding name.

Since all of the relevant data are categorical, CLT does not apply.

The task for this assignment will be to construct a contingency table and calculate a $X$^2 statistic and a p-value for that statistic.

Prior to beginning, I opened this dataset in Stata 11 and ran the a contingency table. The results show a statistically significant dependency between call backs and race.

### Hypotheses

* The standard null hypothesis (H0) for a contingency table is that there is no relationship between the two variables.
* The alternative hypothesis (HA) states that the two variables are dependent on each other.

### Analysis
1. Contingency Table
2. $X$^2 stat
3. Margin of error
4. Confidence interval for the chi2 statistic

### Item 1: Contingency Table

In [5]:
# Construct a contingency table using race and callback.
# pandas offers a function to directly create a contingency table

cont_table = pd.crosstab(data.race, data.call)
cont_table.columns = ['No','Yes']
cont_table.index = ['Black','White']
cont_table

,No,Yes
Black,2278,157
White,2200,235


### Item 2: Chi^2 and p-value

In [6]:
# Calculate a chi^2 statistics, p-value
# scipy, in its stats submodule, offers a function to calculate these statistics and perform this test

#chi2, p, df, ex: chi2 stat, p value, deg. of freedom, and expected table
chi2, p, df, ex = stats.chi2_contingency(cont_table)

print("Test statistic:", chi2)
print("p-value:",p)

Test statistic: 16.4490285842
p-value: 4.99757838996e-05


We have sufficient information to draw a conclusion at this point: the racial appearance of a name and callback status are dependent, while the table suggests that black-sounding names are called back less frequently.

The assignment still calls for confidence intervals and margin of error, and so I am proceeding to produce these statistics.

### Items 3 and 4: Replace Margin of Error and Confidence Interval with Critical Value

$X$^2 distributions are bounded at 0 and have a tail that approaches the x-axis, the larger the x value is. Tests using a $X$^2 distribution use a one-way test, requiring higher values to prove statistical significance. As a result, there is one critical value, rather than a broader confidence interval that encapsulates a margin of error for an estimate. I will calculate this critical value instead of a margin of error and confidence interval. This result is then compared with the $X$^2 statistic calculated above.

In [7]:
# calculate the critical value 95% confidence
chi2_crit = stats.chi2.ppf(q = 0.95, df = df)
print("Chi-square Critical value:", chi2_crit)
print("The observed statistic is greater than the critical value: %s." % str(chi2 > chi2_crit))

Chi-square Critical value: 3.84145882069
The observed statistic is greater than the critical value: True.


The data repeatedly show a clear conclusion: if your resume has an African-American sounding name on it, you are less likely to be called back by a potential employer. Tests of independence between what one's name sounds like and receiving a call back fail; these data support rejecting the null hypothesis of independence, with a clear p-value result, well below the 5% threshold needed to reject the null hypothesis.

The results do not indicate how large the effect is. Additional analysis, most likely a linear regression, is needed to measure the impact size. It is also not clear if the racial appearance of the name is the most important factor, and, indeed, one at least hopes that it is not. The low rate of callback for all individuals in the survey does suggest that some other factor(s) may be more important. Additional independent variables and linear regression is likely the best tool to further explore the relationship between the racial appearance of an applicant's name and that applicant's likelihood of receiving a callback.

Nevertheless, the fundamental research question of this analysis, and the reason for gathering this data, was to determine if race impacts an applicant's chance of receiving a callback. The contingency table results, if all the other potential factors have been properly controlled in the survey sample, should be sufficient for drawing general conclusions. And these conclusions  indicate that an applicant's race is a factor in determining whether or not the applicant receives a callback from a potential employer.